In [1]:
# Thanks for Code from https://www.kaggle.com/mirzarahim/introduction-to-pca-image-compression-example
# Introduction to PCA: Image Compression example

# https://github.com/vivekrmk/Image-Compression-Principal-Component-Analysis-Pytorch/blob/main/Pytorch_PCA_journey.ipynb
# https://github.com/Erikfather/PCA-python/blob/master/Face_Rec.py

In [1]:
# Checking GPU Units

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import collections
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

True
NVIDIA GeForce RTX 2060
1


In [8]:
# DATA_PATH = '../../../Documents/shopee-product-matching/'
DATA_PATH = 'shopee-product-matching/'
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.image import imread
from IPython.display import Image
from PIL import Image
import tensorflow as tf
from tqdm import tqdm

train = pd.read_csv(DATA_PATH + 'train.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')
train['image'] = DATA_PATH + 'train_images/' + train['image']
test['image'] = DATA_PATH + 'test_images/' + test['image']

n_train = 1200
n_test = 3000
n_valid = 300
K = 500 # PCA, num of principal components

sample = train.head(n_train)
tmp = sample.groupby('label_group').posting_id.agg('unique').to_dict()
sample['target'] = sample.label_group.map(tmp)
#test   = train.loc[n_train+1:n_train+n_test]
#test = test.reset_index(drop=True) # initialize indexing
image_idx = sample['image']



<ipython-input-8-906f0233b0dd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['target'] = sample.label_group.map(tmp)


In [83]:
test
test_idx = test['image']

In [4]:
sample

,posting_id,image,image_phash,title,label_group,target
0,train_129225211,shopee-product-matching/train_images/0000a6881...,94974f937d4c2433,Paper Bag Victoria Secret,249114794,[train_129225211]
1,train_3386243561,shopee-product-matching/train_images/00039780d...,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,[train_3386243561]
2,train_2288590299,shopee-product-matching/train_images/000a190fd...,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,[train_2288590299]
3,train_2406599165,shopee-product-matching/train_images/00117e4fc...,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]"
4,train_3369186413,shopee-product-matching/train_images/00136d1cf...,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,[train_3369186413]
...,...,...,...,...,...,...
11995,train_546357643,shopee-product-matching/train_images/5a4ff63df...,afe0f468ca9f9881,Skipping Jump Rope. Skipping Jump Digital Counter,47068657,"[train_779610129, train_546357643]"
11996,train_3553830288,shopee-product-matching/train_images/5a586ca7c...,e5969a49cc3633cc,[ TOPLES ] Kenko PC-3200JR Paper Clip - Warna ...,3403785568,[train_3553830288]
11997,train_2304804397,shopee-product-matching/train_images/5a5b56405...,ab618d3b91740d3e,Handgrip handfat grip karbon karet kitako ktc ...,474797793,[train_2304804397]
11998,train_2376848499,shopee-product-matching/train_images/5a5f2a56b...,c1b78fc375c70238,(Lochic) L-01 Sepatu LOL LED Anak Terbaru Impo...,4008024672,"[train_2376848499, train_1934823359]"


In [84]:
test

,posting_id,image,image_phash,title
0,test_2255846744,shopee-product-matching/test_images/0006c8e546...,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...
1,test_3588702337,shopee-product-matching/test_images/0007585c4d...,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...
2,test_4015706929,shopee-product-matching/test_images/0008377d36...,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng
3,test_1122,shopee-product-matching/test_images/aaaaa.jpg,1a1a1a1a,phone


In [3]:
def getF1score(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

def getPrecision(col): # col = oof_cnn
    def precision(row):
        
        a = np.in1d(row.target,row[col])
        temp = collections.Counter(a)
        correct = temp[True]/len(a)
        
        # if np.where(row.oof_cnn == row.target[0]) != []:
        #     correct = 1
        # else:  
        #     correct = 0

        return correct

    return precision

def getRecall(col):
    def recall(row):
        return 1/len(row[col])
    return recall

In [12]:
class ImageTransform():
    def __init__(self, mean, std):
        self.data_transform = transforms.Compose([
            transforms.Resize((512,512)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    def __call__(self, img):
        return self.data_transform(img)

In [13]:
class Img_Dataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img = img.convert('L')
        img_transformed = self.transform(img)
        
        return img_transformed

In [14]:
# Preparing train dataset, 2021. 5. 19
train_img_list = image_idx

mean = (0.0,)
std = (1.0,)

train_dataset = Img_Dataset(file_list = train_img_list,
                            transform=ImageTransform(mean, std))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=False)

batch_iterator = iter(train_dataloader)
images = next(batch_iterator)

print(images.size())

# torch.Size[Batch Size, Channel, Width, Height]

torch.Size([1, 1, 256, 256])


In [85]:
# prepare test features 2021. 5. 19
test_image_idx = test['image']
test_img_list = test_image_idx

test_dataset = Img_Dataset(file_list = test_img_list,
                            transform=ImageTransform(mean, std))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

batch_iterator = iter(test_dataloader)
test_images = next(batch_iterator)

len(batch_iterator)
#print(test_images.size())

1

In [23]:
# Making **train** features, 2021. 5. 19 
DEVICE = 'cuda'

train_feature = []
train_feature = torch.tensor(train_feature)

# transfer to DEVICE (GPU memory)
train_feature = train_feature.to(DEVICE)

a = 1
with torch.no_grad():
    for batch in tqdm(train_dataloader):
        batch = batch.to(DEVICE)
        batch = batch.permute(0,2,3,1)[:,:,:,0]
        idx, row, col = batch.shape
        batch = batch.view([len(batch),-1])

        U,S,V = torch.pca_lowrank(batch, q=len(batch), center=True, niter=3)
        V = torch.tensor(V)
        #print('U shape : ', U.shape)
        #print('S shape : ', S.shape)
        #print('V shape : ', V.shape)
        train_feature = torch.cat([train_feature, V.T[:,:K]], dim = 0)
        #print('train_feature shape: ', train_feature.shape)
        
        # For debugging (breaking)
        #if a == 3:
        #    break
        #print('iter num : ', a)
        a = a + 1
        

  0%|                                                                                         | 0/1200 [00:00<?, ?it/s]<ipython-input-23-0ce96b9613de>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  V = torch.tensor(V)
  3%|██▏                                                                             | 32/1200 [00:03<02:05,  9.31it/s]


KeyboardInterrupt: 

In [11]:
train_feature.shape

torch.Size([12000, 1000])

In [12]:
# Saving **train** Features 2021. 5. 19
train_feature = train_feature.data.cpu().numpy()
np.savetxt('trained_feature.csv', train_feature, delimiter=",")

In [13]:
# Loading **train** Features 2021. 5. 19
train_feature = np.loadtxt('trained_feature.csv', delimiter=",")
train_feature = torch.from_numpy(train_feature)
train_feature = train_feature.to(DEVICE)

# l2 norm to kill all the sim in 0-1   ** train_feature
from sklearn.preprocessing import normalize
train_feature = train_feature.data.cpu().numpy()
train_feature = np.vstack(train_feature)
train_feature = normalize(train_feature)
train_feature = torch.from_numpy(train_feature)
train_feature = train_feature.to(DEVICE)

In [92]:
# Making **test** features, 2021. 5. 19 
DEVICE = 'cuda'

test_feature = []
test_feature = torch.tensor(test_feature)

# transfer to DEVICE (GPU memory)
test_feature = test_feature.to(DEVICE)

a = 1
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        batch = batch.to(DEVICE)
        #print('degug(batch shape) : ', batch.shape)
        batch = batch.permute(0,2,3,1)[:,:,:,0]
        idx, row, col = batch.shape
        batch = batch.view([len(batch),-1])

        U,S,V = torch.pca_lowrank(batch, q=len(batch), center=True, niter=3)
        V = torch.tensor(V)
        #print('U shape : ', U.shape)
        #print('S shape : ', S.shape)
        #print('V shape : ', V.shape)
        
        test_feature = torch.cat([test_feature, V.T[:,:K]], dim = 0)
        #print('test_feature shape: ', test_feature.shape)
        
        # For debugging (breaking)
        #if a == 3:
        #    break
        #print('iter num : ', a)
        a = a + 1
        

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

degug(batch shape) :  torch.Size([4, 1, 512, 512])


<ipython-input-92-e96b2c708ec7>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  V = torch.tensor(V)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.71it/s]


In [87]:
# Saving **test** Features 2021. 5. 19
test_feature = test_feature.data.cpu().numpy()
np.savetxt('test_feature.csv', test_feature, delimiter=",")

In [88]:
# Loading **test** Features 2021. 5. 19
test_feature = np.loadtxt('test_feature.csv', delimiter=",")
test_feature = torch.from_numpy(test_feature)
test_feature = test_feature.to(DEVICE)

# l2 norm to kill all the sim in 0-1    ** test_feature
test_feature = test_feature.data.cpu().numpy()
test_feature = np.vstack(test_feature)
test_feature = normalize(test_feature)
test_feature = torch.from_numpy(test_feature)
test_feature = test_feature.to(DEVICE)

In [64]:
# Checking train_feature with train_feature, 2021. 5. 19
preds = []
CHUNK = 100

print('Finding similar images...')
CTS = len(train_feature)//CHUNK
if len(train_feature)%CHUNK != 0:
    CTS += 1
    
for j in tqdm(range(CTS)):
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b, len(train_feature))
    #print('chunk', a, 'to', b)
    
    #distances = torch.cdist(train_feature, train_feature[a:b], p=2.0).T
    distances = torch.matmul(train_feature, train_feature[a:b].T).T
    distances = distances.data.cpu().numpy()
    
    #print(type(distances))
    #print(distances.shape)
    '''
    for k in range(b-a):
        IDX = np.argmin(distances[k][:])
        o = sample.iloc[IDX].label_group
        preds.append(o)
    '''
    
    for k in range(b-a):
        #IDX = np.argmax(distances[k][:])
        IDX = np.where(distances[k,]>0.98)[0][:]
        #IDX = np.where(distances[k,]<0.1)[0][:]
        #o = sample.iloc[IDX].label_group.values
        o = sample.iloc[IDX].posting_id.values
        preds.append(o)
        #print(len(IDX))
    
sample['predicted_label'] = preds


  3%|██▋                                                                               | 4/120 [00:00<00:06, 19.28it/s]

Finding similar images...


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:05<00:00, 20.46it/s]
<ipython-input-64-da2d257f8a70>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['predicted_label'] = preds


In [33]:
distances

array([[1.84136441e+00, 1.69961600e+00, 7.03142078e-01, ...,
        1.10858319e+00, 1.40383329e+00, 1.44742663e+00],
       [8.14778854e-01, 1.07764920e+00, 1.77934641e+00, ...,
        1.11576365e+00, 1.41985635e+00, 1.42193139e+00],
       [1.20254826e+00, 1.23725036e+00, 1.66965049e+00, ...,
        1.82141639e+00, 1.41708337e+00, 1.41142104e+00],
       ...,
       [1.53447938e+00, 1.55071492e+00, 1.22161698e+00, ...,
        0.00000000e+00, 1.41496311e+00, 1.37124155e+00],
       [1.42225938e+00, 1.40857082e+00, 1.40687874e+00, ...,
        1.41496311e+00, 3.65002415e-08, 1.38613575e+00],
       [1.42655339e+00, 1.45180531e+00, 1.40454195e+00, ...,
        1.37124155e+00, 1.38613575e+00, 0.00000000e+00]])

In [65]:
# Scoring 
# 2021. 5. 20 edited

sample['f1'] = sample.apply(getF1score('predicted_label'),axis=1)
print('CV score for baseline = ', sample.f1.mean())
sample['Prec'] = sample.apply(getPrecision('predicted_label'),axis=1)
print('precision = ', sample.Prec.mean())
sample['Rec'] = sample.apply(getRecall('predicted_label'),axis=1)
print('recall = ', sample.Rec.mean())

# Score history --------------------------------
# 2021. 5. 20, K = 200, distances>0.95
# F1 = 0.33
# Pr = 0.45
# Re = 0.56

# 2021. 5. 20, K = 1000, samples = 12000, l2 distance < 0.1
# CV score for baseline =  0.66716788743467
# precision =  0.6060484848484848
# recall =  0.9470901533062724

# 2021. 5. 20, K = 1000, samples = 12000, cos-similarity > 0.98
# CV score for baseline =  0.5838659852896299
# precision =  0.6078847164137605
# recall =  0.8174100792345147

<ipython-input-65-1e2c692ef894>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['f1'] = sample.apply(getF1score('predicted_label'),axis=1)


CV score for baseline =  0.5838659852896299
precision =  0.6078847164137605
recall =  0.8174100792345147


<ipython-input-65-1e2c692ef894>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['Prec'] = sample.apply(getPrecision('predicted_label'),axis=1)
<ipython-input-65-1e2c692ef894>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['Rec'] = sample.apply(getRecall('predicted_label'),axis=1)
